In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from tqdm import tqdm

import myfunctions
import myplot
import mylearn
import sklearn.cluster
import sklearn.mixture
from sklearn.metrics.cluster import homogeneity_completeness_v_measure
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

logitech_step1_fast = ('logitech-fast/niels-step_1-1617765899871', 0)
logitech_step1_slow = ('logitech-slow/niels-step_1-1617765658270', 60, True)
lenovo_slow = ('lenovo-slow-easy/niels-step_5-1617952576650', 0)
lenovo_slow2 = ('lenovo-slow-easy/niels-step_5-1618197371437', 0)
lenovo_hard = ('lenovo-hard/niels-step_1-1618199054005', 0)
noweb1 = ('winform/winform-1618904428438', 60, False)

In [ ]:
path, sync_adjustment, adjust_missing_samples = logitech_step1_slow
d = myfunctions.getData(path, sync_adjustment=sync_adjustment, adjust_missing_samples=adjust_missing_samples)

In [ ]:
# myplot.plotKeystrokeContext(d, 11, 50)

# for i in range(21, 31):
#     myplot.plotKeystroke(d, i)

myplot.plotLostSamples(d)

In [ ]:
mylearn.addFeatures(d)

In [ ]:

def tmp(data):
    def filterLabels(labels, keep=['Space', 'Backspace', 'Enter']):
        return [label if label in keep else 'Other' for label in labels]
    
    labels = data['keystroke_labels']
    print(len(labels))
    features = mylearn.getConcatenatedFeatures(data, ['mfcc_mean', 'mfcc_std'])
    
    filtered_labels = filterLabels(labels, keep = ['Space', 'Back', 'Return'])
    n_clusters = len(set(filtered_labels)) + 1

    clustering=sklearn.cluster.k_means(features, n_clusters=n_clusters)[1]
    mylearn.printClusteringResult(clustering, filtered_labels, 'k_means')

    gm = sklearn.mixture.GaussianMixture(n_clusters)
    gm.fit(features)
    clustering = gm.predict(features)
    mylearn.printClusteringResult(clustering, filtered_labels, 'GaussianMixture')
tmp(d)


In [ ]:
def findNumberOfClusters(features):
    x = list(range(2, 20))
    scores = []
    for n_clusters in tqdm(x):
        kmeans = sklearn.cluster.KMeans(n_clusters)
        kmeans.fit(features)
        kmeans_score = kmeans.score(features)
        clustering = kmeans.predict(features)
        kmeans_silhouette_score = silhouette_score(features, clustering)

        gm = sklearn.mixture.GaussianMixture(n_clusters)
        gm.fit(features)
        gm_score = gm.score(features)
        gm_bic_score = gm.bic(features)
        
        scores.append((kmeans_score, kmeans_silhouette_score, gm_score, gm_bic_score))
    return x, scores

features = mylearn.getConcatenatedFeatures(d, ['mfcc_max', 'mfcc_mean', 'mfcc_std'])
number_of_clusters,scores = findNumberOfClusters(mylearn.getConcatenatedFeatures(d, ['mfcc_max', 'mfcc_mean', 'mfcc_std']))

for (i, s) in enumerate(['kmeans_score', 'kmeans_silhouette_score', 'gm_score', 'gm_bic_score']):
    plt.figure()
    plt.title(s)
    plt.scatter(number_of_clusters, [x[i] for x in scores])
    plt.show()

In [ ]:
wav = logitech_step1_slow['keystroke_wavs'][0]
print( librosa.feature.spectral_centroid(wav, 48000, win_length=1000, hop_length=2).shape )
print( librosa.feature.chroma_stft(wav, 48000, win_length=1000, hop_length=2).shape )
print( librosa.feature.zero_crossing_rate(wav, frame_length=1000, hop_length=2).shape )
print( list(librosa.feature.zero_crossing_rate(wav, frame_length=2, hop_length=2)[0]) )

In [ ]:
# getDroppedSamplesTimeAndLength(logitech_step1_fast, True)
# getDroppedSamplesTimeAndLength(logitech_step1_slow, True)
# plotBokeh(getDataSubset(logitech_step1_slow, 1800, 3000), 50)
# plotBokeh(getDataSubset(logitech_step1_slow, 142000, 2000), 50, whichWav='wav')
# plotBokeh(getDataSubset(logitech_step1_slow, 142000, 2000), 50, whichWav='adjusted_wav')
# plotBokeh(getDataSubset(logitech_step1_slow, 182000, 2000), 50, whichWav='wav')
# plotBokeh(getDataSubset(logitech_step1_slow, 182000, 2000), 50, whichWav='adjusted_wav')
# plotBokeh(getDataSubset(logitech_step1_slow, 0, 2000), 50, whichWav='wav')
# getDroppedSamplesTimeAndLength(lenovo_slow, True)

for i in range(10, 17):
    myplot.plotKeystroke(d, i)